An initial attempt at NWB conversion for NeuraLynx data following "manual" conversion described in https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html .  Unlike the example(s) there I (Yarik) was trying to identify levels of data and metadata to consider, and also to store them across multiple .nwb files. 

In [15]:
import sys
sys.path.append('..')

import numpy as np
import pynwb
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile
from conv.io import MovieParadigm
import os.path as op
import re
import glob

import neo

In [16]:
# session_data = '/Users/XinNiuAdmin/Documents/NWBTest/D550/EXP1_Screening/2022-03-10_13-34-18'

directory_path = '/Volumes/DATA/NLData/D570/EXP5_Movie_24_Sleep/2024-01-27_00-01-35/'
# Define the pattern
pattern = 'G[A-D][1-9]*.ncs'

# Use glob to find files matching the pattern in the specified directory
session_data = glob.glob(directory_path + pattern) 
session_data = session_data[0:6]
session_data

['/Volumes/DATA/NLData/D570/EXP5_Movie_24_Sleep/2024-01-27_00-01-35/GA1-REC1.ncs',
 '/Volumes/DATA/NLData/D570/EXP5_Movie_24_Sleep/2024-01-27_00-01-35/GA1-REC1_0002.ncs',
 '/Volumes/DATA/NLData/D570/EXP5_Movie_24_Sleep/2024-01-27_00-01-35/GA1-REC1_0003.ncs',
 '/Volumes/DATA/NLData/D570/EXP5_Movie_24_Sleep/2024-01-27_00-01-35/GA1-REC1_0004.ncs',
 '/Volumes/DATA/NLData/D570/EXP5_Movie_24_Sleep/2024-01-27_00-01-35/GA1-REC1_0005.ncs',
 '/Volumes/DATA/NLData/D570/EXP5_Movie_24_Sleep/2024-01-27_00-01-35/GA1-REC1_0006.ncs']

In [17]:
# Common lab wide metadata
lab_metadata = dict(
    lab="CNL",
    institution="UCLA Health",
    keywords=["DANDI Pilot"], # arbitrary, so let's promote!
)
# Experiment specific one
experiment_metadata = dict(
    experimenter="Xin Niu <michaelniki1988@gmail.com>",  # Let's see if nwb can swallow such a record ;)
    experiment_description="movie paradigm",
)


In [18]:
# create a reader
reader = neo.io.NeuralynxIO(include_filename=session_data) # TODO: newer version should support: , keep_original_times=True)
reader.parse_header()
print(reader)

NeuralynxIO: 
nb_block: 1
nb_segment:  [328]
signal_streams: [Stream (rate,#packet,t0): (32000.0, 450001, 1706336056470695) (chans: 1), Stream (rate,#packet,t0): (32000.0, 450001, 1706328856470711) (chans: 1), Stream (rate,#packet,t0): (32000.0, 450001, 1706321656470695) (chans: 1), Stream (rate,#packet,t0): (32000.0, 450001, 1706314456470711) (chans: 1), Stream (rate,#packet,t0): (32000.0, 50276, 1706343256470710) (chans: 1)]
signal_channels: [GA1-REC1, GA1-REC1, GA1-REC1, GA1-REC1, GA1-REC1]
spike_channels: []
event_channels: []



In [19]:
# seg = reader.read_segment()
seg = reader.read()

In [21]:

filename_metadata = re.match(
    r'.*/(?P<subject_id>D\d+)/(?P<experiment>EXP\d+_[\w\d]+)/(?P<date>\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2})/',
    directory_path,
).groupdict()
assert filename_metadata
filename_metadata

{'subject_id': 'D570',
 'experiment': 'EXP5_Movie_24_Sleep',
 'date': '2024-01-27_00-01-35'}

In [22]:
import time

# Those time stamps are in sub-second and not the one we would want to the "session time"
# time.gmtime(reader.get_event_timestamps()[0][0])
# TODO: figure out where in this 
# TODO: figure out what those timestamps in.

In [26]:
# Metadata which is likely to come from data files and "promotion" metadata records

# Most likely many could be parsed from the filenames which are likely to encode some of it
# So "heuristical" converter could establish metadata harvesting from the filenames

#
# Session specific
session_metadata = dict(
    session_id="%(subject_id)s-%(date)s" % filename_metadata,
    session_description="Extracellular ephys recording in the left hemisphere of the nucleus accumbens",  # args[0] in nwbfile
    session_start_time=datetime.now(tzlocal()), # TEMP  # args[2] in nwbfile; TODO needs to be datetime
)
subject_metadata = dict(
    subject_id=filename_metadata['subject_id'],
    weight='',
    age="TODO",  # duplicate with session_start_time and date_of_birth but why not?
    species='',
    sex="female",
#     hemisphere=metadata_keys['ExpKeys.hemisphere'],
#     depth=metadata_keys['ExpKeys.probeDepth'],
#     region=metadata_keys['ExpKeys.target'],
    date_of_birth=datetime.now(tzlocal()), # TEMP: TODO
)
surgery_metadata = dict(
    surgery="Headbar on xx/xx/2020, craniotomy over right hemisphere on xx/xx/2020, craniotomy over left hemisphere on xx/xx/2020. All surgeries performed by JG."
)
# Actually probably only "identifier" should be file specific, the rest common across files
# we would like to produce: separate for .ncs, .ntt, behavioral metadata, etc
file_metadata = dict(
    source_script="somescript-not-clear-whyneeds to be not empty if file_name is provided",
    source_script_file_name="TODO", # __file__,
)

# common filename prefix - let's mimic DANDI filenaming convention right away
filename_prefix = "sub-{subject_id}_ses-{session_id}".format(**subject_metadata, **session_metadata)
# the rest will be specific to the corresponding file. E.g. we will have separate
#  - `_probe-<name>_ecephys.nwb` (from each .ncs) - contineous data from each tetrode. probably chunked and compressed
#  - `_???_ecephys.nwb` (from each .ntt) - spike detected windowed data. 
#  - `_behav.mpg` + `_behav.nwb` - video recording and metadata (including those .png?) for behavior component within experiment recording session
# Pretty much we need to establish a framework where EVERY file present would be
# provided

print(subject_metadata)

# TODO add event labels to metadata

{'subject_id': 'D570', 'weight': '', 'age': 'TODO', 'species': '', 'sex': 'female', 'date_of_birth': datetime.datetime(2024, 3, 21, 19, 17, 48, 251807, tzinfo=tzlocal())}


In [27]:
# Such NWBFile will be created for each separate file, and then fill up with the corresponding
#
filename_suffix = "TODO"
nwbfile = NWBFile(
    identifier="{}_{}".format(filename_prefix, filename_suffix), # args[1] in nwbfile, may be just UUID? not sure why user has to provide it really
    subject=pynwb.file.Subject(**subject_metadata),
    **lab_metadata,
    **experiment_metadata,
    **session_metadata,
    **surgery_metadata,
    **file_metadata,)

print(nwbfile.identifier)

sub-D570_ses-D570-2024-01-27_00-01-35_TODO


In [32]:
dir(reader)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_check_stream_signal_channel_characteristics',
 '_empty_ncs',
 '_empty_nev',
 '_empty_nse_ntt',
 '_event_count',
 '_generate_minimal_annotations',
 '_get_analogsignal_chunk',
 '_get_channel_indexes',
 '_get_event_timestamps',
 '_get_file_map',
 '_get_signal_size',
 '_get_signal_t_start',
 '_get_spike_raw_waveforms',
 '_get_spike_timestamps',
 '_get_stream_index_from_arg',
 '_nb_segment',
 '_ncs_dtype',
 '_nev_memmap',
 '_parse_header',
 '_prefered_signal_group_mode',
 '_repr_annotations',
 '_rescale_epoch_duration',
 '_rescale_event_timestamp',
 '_rescale_spike_timestamp',
 '_seg_t_starts',
 '_seg_t_stops',
 '

In [28]:
# add electrode metadata
# create probe device
device = nwbfile.create_device(name='micro probe', description='', manufacturer='NeuraLynx')

# for each channel on the probe
for chl in reader.header['unit_channels']:
    
    # get tetrode id
    tetrode = re.search('(?<=TT)(.*?)(?=#)', chl[0]).group(0)
    electrode_name = 'tetrode' + tetrode
    
    # get channel id
    channel = re.search('(?<=#)(.*?)(?=#)', chl[0]).group(0)
           
    if electrode_name not in nwbfile.electrode_groups: # make tetrode if does not exist
    
        description = electrode_name
        location = metadata_keys['ExpKeys.hemisphere'] + ' ' + metadata_keys['ExpKeys.target'] + ' ' + \
            '(' + metadata_keys['ExpKeys.probeDepth'] + ' um)'

        electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                         description=description,
                                                         location=location,
                                                         device=device)
        
    # add channel to tetrode
    nwbfile.add_electrode(id=int(channel),
                          x=-1.2, y=float(metadata_keys['ExpKeys.probeDepth']), z=-1.5,
                          location=metadata_keys['ExpKeys.target'], filtering='none',
                          imp = 0.0, group=nwbfile.electrode_groups[electrode_name])

KeyError: 'unit_channels'

In [ ]:
# append data from different segments

spk_all = []
wv_all = []
csc_all_mag = []
csc_all_time = [];
beh_all = []

for s in range(reader.header['nb_segment'][0]):
#     for i, chl in enumerate(reader.header['unit_channels']):
    if s is 0:
        for i, chl in enumerate(seg[0].segments[s].spiketrains):
            spk_all.append([seg[0].segments[s].spiketrains[i].times])
            
        csc_all_mag = seg[0].segments[s].analogsignals[0].magnitude
        csc_all_time = seg[0].segments[s].analogsignals[0].times
        
        for i, chl in enumerate(seg[0].segments[s].events):
            beh_all.append([seg[0].segments[s].events[i].times])
            
    else:
        for i, chl in enumerate(seg[0].segments[s].spiketrains):
            spk_all[i] = np.append(spk_all[i],[seg[0].segments[s].spiketrains[i].times])
            
        csc_all_mag = np.vstack([csc_all_mag,seg[0].segments[s].analogsignals[0].magnitude])
        csc_all_time = np.append(csc_all_time,seg[0].segments[s].analogsignals[0].times)
        
        for i, chl in enumerate(seg[0].segments[s].events):
            beh_all[i] = np.append(beh_all[i],seg[0].segments[s].events[i].times)

In [ ]:
# add data to nwb file
from pynwb.ecephys import ElectricalSeries
from pynwb.ecephys import SpikeEventSeries
from pynwb.ecephys import EventWaveform
from pynwb.behavior import BehavioralTimeSeries

# add .ntt files
ephys_waveform = EventWaveform()

# loop through .ntt files
for i, chl in enumerate(reader.header['unit_channels']):
    
    # get tetrode id
    tetrode = re.search('(?<=TT)(.*?)(?=#)', chl[0]).group(0)
    tetrode_name = 'TT' + tetrode
           
    if tetrode_name not in ephys_waveform.spike_event_series: # make tetrode if does not exist
        
        chl_list = []
        
        for j, group in enumerate(nwbfile.electrodes['group']):
        
            if tetrode in nwbfile.electrodes['group'][j].fields['description']:
                
                chl_list.append(j)
        
        electrode_table_region = nwbfile.create_electrode_table_region(chl_list, tetrode_name)
        
        for s in range(reader.header['nb_segment'][0]):
            
            if s is 0:
                
                waveform = reader.get_spike_raw_waveforms(seg_index=s, unit_index=i)
                
            else:
                
                waveform = np.vstack([waveform,reader.get_spike_raw_waveforms(seg_index=s, unit_index=i)])

        ephys_waveform.create_spike_event_series(tetrode_name,
                                                 waveform,
                                                 spk_all[i],
                                                 electrode_table_region)

nwbfile.add_acquisition(ephys_waveform)

# add .ncs files

chl_list = []

for chl in reader.header['signal_channels']['id']:
    
    chl_list.append(nwbfile.electrodes['id'][:].index(chl))
    
electrode_table_region = nwbfile.create_electrode_table_region(chl_list, 'CSC order for time series')

ephys_ts = ElectricalSeries('CSC data',
                            csc_all_mag,
                            electrode_table_region,
                            timestamps=csc_all_time,
                            comments='n/a',
                            description='unfiltered CSC data')

nwbfile.add_acquisition(ephys_ts)

# nwbfile.add_unit(id=1, electrodes=[0])
# nwbfile.add_unit(id=2, electrodes=[0])

# add .evt file

beh_ts = BehavioralTimeSeries()

# loop through events
for i, chl in enumerate(reader.header['event_channels']):
        
    beh_ts.create_timeseries(str(chl),
                                timestamps = beh_all[i])

nwbfile.add_acquisition(beh_ts)


In [ ]:
# Save the generated file
from pynwb import NWBHDF5IO

# TODO: I think we should right away use dandi-cli provided API to create the filename based on metadata
# in the NWBFile
with NWBHDF5IO('BCD_example.nwb', 'w') as io:
    io.write(nwbfile)